In [1]:
import pandas as pd
import requests
import json

In [2]:
API_KEY = API_KEY

In [3]:
client_id = 'ZPsDRbHzglthukFTIg4SEA'

In [4]:
headers = {'Authorization': 'Bearer %s' % API_KEY}
url='https://api.yelp.com/v3/businesses/search'

In [5]:
zip_codes_SF = [94102, 94104, 94103, 94105, 94108, 94107, 94110, 
                94109, 94112, 94111, 94115, 94114, 94117, 94116,
                94118, 94121, 94123, 94122, 94124, 94127, 94126,
                94129, 94131, 94130, 94133, 94132, 94134, 94139,
                94143, 94151, 94159, 94158, 94188, 94177, 94132,
                94122, 94116, 94124, 94121, 94118, 94117, 94114, 
                94124, 94107, 94133, 94108]

In [6]:
def get_values(key, list_dicts):
    values = []
    for dict_ in list_dicts:
        try:
            values.append(dict_[key])
        except:
            values.append(None)         
    return values

In [7]:
ids, names, rating, review_count, price, location, coordinates, transactions, categories, is_closed = [],[],[],[],[],[],[],[],[],[]
for zip_code in zip_codes_SF:
    params = {'term':'restaurants', 'location': str(zip_code), 'limit':'50'}
    req=requests.get(url, params=params, headers=headers)
    
    # proceed only if the status code is 200
    print('The status code is {}'.format(req.status_code))
    
    parsed = json.loads(req.text)
    try:
        businesses = parsed['businesses']


        ids.append(get_values('id', businesses))
        names.append(get_values('name', businesses))
        rating.append(get_values('rating', businesses))
        review_count.append(get_values('review_count', businesses))
        price.append(get_values('price', businesses))
        location.append(get_values('location', businesses))
        coordinates.append(get_values('coordinates', businesses))
        transactions.append(get_values('transactions', businesses))
        categories.append(get_values('categories', businesses))
        is_closed.append(get_values('is_closed', businesses))
    except:
        pass

The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status 

In [8]:
list_attrs = [ids, names, rating, review_count,
             price, transactions, is_closed]
name_attrs = ['ids', 'names', 'rating', 'review_count',
             'price', 'transactions', 'is_closed']

In [9]:
dict_flat = {}
for i, attrs in enumerate(list_attrs):
    dict_flat[name_attrs[i]] = [x for attr in attrs for x in attr]

In [10]:
alias_list = []
title_list = []

for category in categories:

    for cat in category:
        alias = []
        title = []
        for c in cat:
            try:
                alias.append(c["alias"])
                title.append(c["title"])
            except:
                pass
        alias_attr = ', '.join(alias)
        title_attr = ', '.join(title)

        alias_list.append(alias_attr) 
        title_list.append(title_attr) 

In [11]:
dict_flat['type_food'] = title_list
dict_flat['type_food_alias'] = alias_list

In [12]:
dict_flat['latitude'] = [x['latitude'] for coord in coordinates for x in coord]
dict_flat['longitude'] = [x['longitude'] for coord in coordinates for x in coord]

In [13]:
dict_flat['country'] = [x['country'] for loc in location for x in loc]
dict_flat['state'] = [x['state'] for loc in location for x in loc]
dict_flat['city'] = [x['city'] for loc in location for x in loc]
dict_flat['zip_code'] = [x['zip_code'] for loc in location for x in loc]

In [14]:
dict_flat.keys()

dict_keys(['ids', 'names', 'rating', 'review_count', 'price', 'transactions', 'is_closed', 'type_food', 'type_food_alias', 'latitude', 'longitude', 'country', 'state', 'city', 'zip_code'])

In [15]:
df = pd.DataFrame.from_dict(data=dict_flat)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2256 entries, 0 to 2255
Data columns (total 15 columns):
ids                2256 non-null object
names              2256 non-null object
rating             2256 non-null float64
review_count       2256 non-null int64
price              1761 non-null object
transactions       2256 non-null object
is_closed          2256 non-null bool
type_food          2256 non-null object
type_food_alias    2256 non-null object
latitude           2256 non-null float64
longitude          2256 non-null float64
country            2256 non-null object
state              2256 non-null object
city               2256 non-null object
zip_code           2256 non-null object
dtypes: bool(1), float64(3), int64(1), object(10)
memory usage: 249.0+ KB


In [17]:
def replace_empty_lists(list_entities):
    if len(list_entities) < 1:
        return None
    return list_entities

In [18]:
df['transactions'] = df.transactions.apply(replace_empty_lists)

In [19]:
df.head()

,ids,names,rating,review_count,price,transactions,is_closed,type_food,type_food_alias,latitude,longitude,country,state,city,zip_code
0,vmZ-bgnM5B76NUpegAjy1g,Hotbird,4.5,129,None,None,False,"Sandwiches, Chicken Shop, Burgers","sandwiches, chickenshop, burgers",37.776800,-122.416264,US,CA,San Francisco,94103
1,kFgjHGp-cTUK5uPaSJOORA,Souvla,4.0,1414,$$,[delivery],False,"Greek, Mediterranean","greek, mediterranean",37.776430,-122.425000,US,CA,San Francisco,94102
2,KA-P3d2NcbJy5SQN-vCzPA,Aria Korean Street Food,4.0,882,$$,"[delivery, pickup]",False,"Korean, Tapas/Small Plates, Chicken Shop","korean, tapasmallplates, chickenshop",37.786550,-122.417950,US,CA,San Francisco,94109
3,AMhxom9yhQjTkFRMTKk_IA,WingSlut,5.0,5,None,None,False,"Chicken Wings, Fast Food, Asian Fusion","chicken_wings, hotdogs, asianfusion",37.776775,-122.416414,US,CA,San Francisco,94103
4,lJAGnYzku5zSaLnQ_T6_GQ,Brenda's French Soul Food,4.0,11054,$$,[delivery],False,"Breakfast & Brunch, Southern, Cajun/Creole","breakfast_brunch, southern, cajun",37.782902,-122.419043,US,CA,San Francisco,94102


In [20]:
df.to_csv('../../data/interim/restaurants/yelp_sanfrancisco.csv', index=False)